
## Arrow and Parquet Files: Answers to Bonus Questions, Thomas (Mark) McCamley 

* Data source: New York City Taxi and Limousine Commission (TLC), month of January 2009.

In [20]:
# Load the resources we need, download the data, which is in parquet file format

import pyarrow as pa 
import pyarrow.compute as pc
import pyarrow.parquet as pq
dataset = pq.ParquetDataset('s3://ursa-labs-taxi-data/2009/', partitioning=["month"])
dataset.files

['ursa-labs-taxi-data/2009/01/data.parquet',
 'ursa-labs-taxi-data/2009/02/data.parquet',
 'ursa-labs-taxi-data/2009/03/data.parquet',
 'ursa-labs-taxi-data/2009/04/data.parquet',
 'ursa-labs-taxi-data/2009/05/data.parquet',
 'ursa-labs-taxi-data/2009/06/data.parquet',
 'ursa-labs-taxi-data/2009/07/data.parquet',
 'ursa-labs-taxi-data/2009/08/data.parquet',
 'ursa-labs-taxi-data/2009/09/data.parquet',
 'ursa-labs-taxi-data/2009/10/data.parquet',
 'ursa-labs-taxi-data/2009/11/data.parquet',
 'ursa-labs-taxi-data/2009/12/data.parquet']

In [21]:
# Create a table with the single month of Janaury file, sort the "pickup_at" and "fare_amount" columns, creating a sorted index object
# Really only need "pickup_at" sorted though

data_table = dataset.fragments[0].to_table()
sorted_indices = pc.sort_indices(data_table, sort_keys=[("pickup_at", "ascending"), ("fare_amount", "ascending")])

In [22]:
# Apply the index object to the table to effect the sorting

sorted_table = data_table.take(sorted_indices)

In [23]:
# As shown below, the sorting is doing what we want with the "pickup_at" timestamp, but two of the three questions are in
# terms of the hour of the day: Can you get the average transaction between 2:00-2:59 PM, and
# Which time of the day has the highest tip?

sorted_table

pyarrow.Table
vendor_id: string
pickup_at: timestamp[us]
dropoff_at: timestamp[us]
passenger_count: int8
trip_distance: float
pickup_longitude: float
pickup_latitude: float
rate_code_id: null
store_and_fwd_flag: string
dropoff_longitude: float
dropoff_latitude: float
payment_type: string
fare_amount: float
extra: float
mta_tax: float
tip_amount: float
tolls_amount: float
total_amount: float
----
vendor_id: [["CMT","CMT","CMT","CMT","CMT",...,"CMT","CMT","CMT","CMT","CMT"]]
pickup_at: [[2009-01-01 00:00:00.000000,2009-01-01 00:00:00.000000,2009-01-01 00:00:02.000000,2009-01-01 00:00:04.000000,2009-01-01 00:00:07.000000,...,2009-01-31 23:59:59.000000,2009-01-31 23:59:59.000000,2009-01-31 23:59:59.000000,2009-01-31 23:59:59.000000,2009-01-31 23:59:59.000000]]
dropoff_at: [[2009-01-01 00:05:03.000000,2009-01-01 00:04:12.000000,2009-01-01 00:05:40.000000,2009-01-01 00:03:08.000000,2009-01-01 00:19:01.000000,...,2009-02-01 00:07:04.000000,2009-02-01 00:06:17.000000,2009-02-01 00:11:54.000000

In [24]:
# To answer these, we'll truncate the timestamp to just hours and minutes so we can isolate single hour blocks,
# and apply it to the table we created to get the new_sorted_table to create new_sorted_table

hour_time = pc.strftime(sorted_table.column("pickup_at"), format='%H:%M')
new_sorted_table = sorted_table.set_column(1, "pickup_hour", hour_time)

In [25]:
# Now we can easily filter this table by hour blocks.  We'll start with 2:00PM - 2:59PM to answer the first question

h_slot = pc.and_(pc.greater_equal(hour_time,'14:00'), pc.less(hour_time, '15:00'))
hour_table = pc.filter(new_sorted_table, h_slot)
hour_table

pyarrow.Table
vendor_id: string
pickup_hour: string
dropoff_at: timestamp[us]
passenger_count: int8
trip_distance: float
pickup_longitude: float
pickup_latitude: float
rate_code_id: null
store_and_fwd_flag: string
dropoff_longitude: float
dropoff_latitude: float
payment_type: string
fare_amount: float
extra: float
mta_tax: float
tip_amount: float
tolls_amount: float
total_amount: float
----
vendor_id: [["VTS","VTS","VTS","VTS","VTS",...,"CMT","DDS","CMT","CMT","CMT"]]
pickup_hour: [["14:00","14:00","14:00","14:00","14:00",...,"14:59","14:59","14:59","14:59","14:59"]]
dropoff_at: [[2009-01-01 14:00:00.000000,2009-01-01 14:02:00.000000,2009-01-01 14:02:00.000000,2009-01-01 14:02:00.000000,2009-01-01 14:03:00.000000,...,2009-01-31 15:05:38.000000,2009-01-31 15:09:39.000000,2009-01-31 15:06:31.000000,2009-01-31 15:17:52.000000,2009-01-31 15:21:42.000000]]
passenger_count: [[1,2,5,5,2,...,1,1,1,2,1]]
trip_distance: [[0.04,0.25,0.64,0.6,0.48,...,1.3,1.1,1.6,1.9,3.5]]
pickup_longitude: [[-73.

### Answer to Bonus Question: Can you get the average transaction between 2:00-2:59 PM?

##### Initially thought that this only referred to fare amount, but since the word "transaction" is used, which implies inclusivity of all charges, I will go with "total_amount"

In [26]:
ave_two_pm = pc.round(pc.mean(hour_table['total_amount']), ndigits=2)
print(f"Answer: the average transaction between 2:00-2:59PM is ${ave_two_pm}")

Answer: the average transaction between 2:00-2:59PM is $10.01


### Answer to Bonus Question: Which time of the day has the highest tip (compute the max value of tip over hourly sections)?

* I interpret this to be asking "what is the highest instance of a tip in each hour slot".  First, the long and complete way

In [27]:
# To compute by hourly sections (via for loop), I only know how to run it using the string tuple list below - clumsy and ugly ;(
# The for loop uses compute logic and values to capture hour start (inclusive), through the 59th minute

hours = [('00:00','01:00'),('01:00','02:00'),('02:00','03:00'),('03:00','04:00'),('04:00','05:00'),('05:00','06:00'),('06:00','07:00'),('07:00','08:00'),('08:00','09:00'),('09:00','10:00'),('10:00','11:00'),('11:00','12:00'),('12:00','13:00'),('13:00','14:00'),('14:00','15:00'),('15:00','16:00'),('16:00','17:00'),('17:00','18:00'),('18:00','19:00'),('19:00','20:00'),('20:00','21:00'),('21:00','22:00'),('22:00','23:00'),('23:00','23:59')]

In [28]:
# The for loop, which filters our table for each hour section, finds the max instance of tip amount in that hour, and then prints for inspection

for j, k in hours:
    h_slot = pc.and_(pc.greater_equal(hour_time,(j)), pc.less(hour_time, (k)))
    hours_table = pc.filter(new_sorted_table, h_slot)
    top_tip = pc.max(hours_table['tip_amount'])
    print(f"Hour beginning {j} has highest instance of tip amount = ${top_tip}")

Hour beginning 00:00 has highest instance of tip amount = $100.0
Hour beginning 01:00 has highest instance of tip amount = $95.44999694824219
Hour beginning 02:00 has highest instance of tip amount = $100.0
Hour beginning 03:00 has highest instance of tip amount = $90.0
Hour beginning 04:00 has highest instance of tip amount = $90.44999694824219
Hour beginning 05:00 has highest instance of tip amount = $90.0
Hour beginning 06:00 has highest instance of tip amount = $80.0
Hour beginning 07:00 has highest instance of tip amount = $100.0
Hour beginning 08:00 has highest instance of tip amount = $83.0
Hour beginning 09:00 has highest instance of tip amount = $89.0
Hour beginning 10:00 has highest instance of tip amount = $81.5
Hour beginning 11:00 has highest instance of tip amount = $100.0
Hour beginning 12:00 has highest instance of tip amount = $100.0
Hour beginning 13:00 has highest instance of tip amount = $100.0
Hour beginning 14:00 has highest instance of tip amount = $100.0
Hour be

##### The highest tip amount of $100 occured in 11 "pick_up" hour slots during the month.  The hundred dollar bill is apparently the max tip vehicle of choice in NYC.

* Now a quicker easier way that shows the number of instances (frequency) of the max value tip over the month, by hour slot

In [29]:
# First find the highest tip amount ...

highest = pc.max(new_sorted_table['tip_amount'])
print(f"The highest tip amount in the month is ${highest}")

The highest tip amount in the month is $100.0


In [30]:
# ... then find which "pickup_hour" slot(s) in which it occurs

max_tip = pc.equal(new_sorted_table['tip_amount'], 100.0)
tip_table = pc.filter(new_sorted_table, max_tip)

# Let's put tip_table into a pandas series and sort

top_tip = tip_table['pickup_hour'].to_pandas()
top_tip.sort_values(ascending=True)

0     00:13
12    00:59
13    02:57
3     07:43
2     11:16
5     12:18
9     13:38
4     14:19
10    16:39
6     19:48
11    20:55
1     21:10
7     21:14
8     21:34
Name: pickup_hour, dtype: object

##### The highest frequency of the max tip amount of $100 occured in the hour slots between 9:00PM and 3:00AM in the morning, so the taxi graveyard shift sees more max value tips.

### Answer to Bonus Question: Which day of the week has the highest average tip amount?

In [31]:
# Let's go back to our original sorted_table, and convert the original timestamp of "pickup_at" to just the day of the week
# First, compute the days of the week as an arrow array from the original timestamps
# The code below sets Monday as the first day of the week.  In 2009, January 1 was a Thursday, so our days shown below start on 4,
# and end on 6, Saturday, the 31st

dow = pa.compute.day_of_week(sorted_table.column("pickup_at"), count_from_zero=False, week_start=1, options=None, memory_pool=None)

# Set the values of the "pickup_at" column according to the array, and name a new table

d_new_sorted_table = sorted_table.set_column(1, "pickup_at", dow)
d_new_sorted_table

pyarrow.Table
vendor_id: string
pickup_at: int64
dropoff_at: timestamp[us]
passenger_count: int8
trip_distance: float
pickup_longitude: float
pickup_latitude: float
rate_code_id: null
store_and_fwd_flag: string
dropoff_longitude: float
dropoff_latitude: float
payment_type: string
fare_amount: float
extra: float
mta_tax: float
tip_amount: float
tolls_amount: float
total_amount: float
----
vendor_id: [["CMT","CMT","CMT","CMT","CMT",...,"CMT","CMT","CMT","CMT","CMT"]]
pickup_at: [[4,4,4,4,4,...,6,6,6,6,6]]
dropoff_at: [[2009-01-01 00:05:03.000000,2009-01-01 00:04:12.000000,2009-01-01 00:05:40.000000,2009-01-01 00:03:08.000000,2009-01-01 00:19:01.000000,...,2009-02-01 00:07:04.000000,2009-02-01 00:06:17.000000,2009-02-01 00:11:54.000000,2009-02-01 00:18:39.000000,2009-02-01 00:26:59.000000]]
passenger_count: [[2,1,1,1,1,...,1,3,2,1,1]]
trip_distance: [[0.9,1.3,1,0.8,5.5,...,1.4,1.8,2,1.3,3]]
pickup_longitude: [[-73.99748,-73.96592,-73.9648,-74.0116,-74.00065,...,-73.977516,-73.97359,-73.99

In [32]:
# Now we can run the same type of loop on the new table, now filtered by the day of the week

day_of_week = range(1, 8, 1)

for i in day_of_week:
    day = pc.equal(dow,(i))
    day_table = pc.filter(d_new_sorted_table, day)
    mean_tip = pc.round(pc.mean(day_table['tip_amount']), ndigits=2)
    print(f"Day of the week {i} has mean tip amount = ${mean_tip}")

Day of the week 1 has mean tip amount = $0.47
Day of the week 2 has mean tip amount = $0.48
Day of the week 3 has mean tip amount = $0.5
Day of the week 4 has mean tip amount = $0.48
Day of the week 5 has mean tip amount = $0.46
Day of the week 6 has mean tip amount = $0.41
Day of the week 7 has mean tip amount = $0.46


##### The highest average tip is on Wednesday, but the day with the highest total tips (sum), by far, is Friday, see below.

In [33]:
for i in day_of_week:
    day = pc.equal(dow,(i))
    day_table = pc.filter(d_new_sorted_table, day)
    sum_tip = pc.round(pc.sum(day_table['tip_amount']), ndigits=2)
    print(f"Day of the week {i} has total tips which sum to = ${sum_tip}")

Day of the week 1 has total tips which sum to = $742213.66
Day of the week 2 has total tips which sum to = $852811.21
Day of the week 3 has total tips which sum to = $891423.74
Day of the week 4 has total tips which sum to = $1102847.63
Day of the week 5 has total tips which sum to = $1142126.72
Day of the week 6 has total tips which sum to = $1036218.4
Day of the week 7 has total tips which sum to = $768644.85
